<a href="https://colab.research.google.com/github/dgromann/TerminologieHilfsmitel/blob/master/LV9_Programmieren.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Um dieses Notebook in Google Colab zu öffnen, klicken Sie bitte direkt unter das Logo in der Mitte über dieser Zeile. Damit sollte sich dann das Notebook in einem neuen Tab in Google Colab öffnen.**

# Lektion 0: Dieses Notebook speichern 

Gehen Sie auf "File" ("Datei") und speichern Sie eine lokale Kopie dieses Notebooks, entweder auf Google Drive oder in Ihrem Github-Konto. Alternativ können Sie das Notebook auch herunterladen und lokal bearbeiten - Achtung dazu muss Python lokal installiert werden und die nachstehenden Code-Abschnitte direkt in Python ausgeführt werden. 

In diesem Notebook finden Sie ausführbaren Programmiercode, den Sie direkt in Ihrem Browser ausführen können. Code ist immer durch einen grauen Rahmen markiert. Wenn Sie in dem grauen Bereich klicken, wird eine "Play" Schalfläche sichtbar.

**Praktische Übungen sind nachstehend als fett gedruckt markiert.**

# Lektion 1: TF-IDF selbst programmieren

Diese Lektion zeigt Ihnen, wie Sie TF-IDF selbst in Python implementieren können. Um alle notwendigen Libraries zu importieren, lassen Sie bitte den nachstehenden Code als erstes laufen.




In [129]:
!pip install stopwords

import nltk
import string
import math

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

lemmatizer = WordNetLemmatizer()
stoplist = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Zuerst erstellen wir einen Beispielkorpus mit unseren drei Sätzen aus der Vorlesung. Wir erstellen die drei Sätze gleich als Liste, damit wir eine Variable haben, die wir bearbeiten (linguistische Vorverarbeitung). 


In [0]:
sentence1 = "Shipment of gold damaged in a fire." 
sentence2= "Delivery of silver arrived in a silver truck." 
sentence3 = "Shipment of gold arrived in a truck."

Im Sinne der linguistischen Vorverarbeitung, werden wir als erste alle Worte klein schreiben. Das können wir in Python automatisch mithilfe der Funktion `lower()` erreichen. 

In [118]:
sentence1 = sentence1.lower()
sentence2 = sentence2.lower()
sentence3 = sentence3.lower()
print(sentence1, "\n", sentence2, "\n", sentence3)

shipment of gold damaged in a fire. 
 delivery of silver arrived in a silver truck. 
 shipment of gold arrived in a truck.


Normalerweise werden weitere Vorverarbeitungsschrittet durchgeführt. Heute bearbeiten wir unsere Sätze wie folgt: 
* Tokenisierung
* Entfernen von Stoppworten 
* Lemmatisierung
* Entfernen von Interpunktion

Dafür definieren wir eine eigene Funktion namens `preprocessing()`. Eine Liste zu ändern die man gerade mit einer `for`-Schleife durchläuft ist meine keine gute Idee. Daher ist es einfacher zu überprüfen ob eine Bedingung nicht erfüllt ist mithilfe von `not` wie folgt dargestellt. Also wenn der Token nicht in der Stoppwortliste ist und auch keine Interpunktion ist, wird er lemmatisiert und der Liste `lemmas` hinzugefügt.

In [119]:
def preprocessing(text): 
  lemmas = []
  tokens = nltk.word_tokenize(text)
  for token in tokens: 
    if token not in stoplist and token not in string.punctuation: 
      lemmas.append(lemmatizer.lemmatize(token))
  return lemmas 

lemmas1 = preprocessing(sentence1)
lemmas2 = preprocessing(sentence2)
lemmas3 = preprocessing(sentence3)
print(lemmas1, "\n", lemmas2, "\n", lemmas3)

['shipment', 'gold', 'damaged', 'fire'] 
 ['delivery', 'silver', 'arrived', 'silver', 'truck'] 
 ['shipment', 'gold', 'arrived', 'truck']


Jetzt müssen wir eine Liste alle Worte in unserem Korpus erstellen. Um Mehrfachnennungen eines Wortes zu elminieren, kann man eine Liste in ein `set` konvertieren. Dazu fügen wir alle Lemma-Listen in eine Liste und konvertieren das ganze in ein Set. 


In [121]:
vocab = []
vocab = lemmas1 + lemmas2
vocab = vocab + lemmas3
print("Wortliste mit Doppelnennungen:", vocab)
print("Wortliste ohne Doppelnennungen:", set(vocab))
unique_vocab = set(vocab)


Wortliste mit Doppelnennungen: ['shipment', 'gold', 'damaged', 'fire', 'delivery', 'silver', 'arrived', 'silver', 'truck', 'shipment', 'gold', 'arrived', 'truck']
Wortliste ohne Doppelnennungen: {'silver', 'truck', 'fire', 'arrived', 'shipment', 'damaged', 'delivery', 'gold'}


In [126]:
def compute_tf(vocab_list):
  """Berechnet die Termhäufigkeit im gesamten Korpus
  auf Basis der gesamten Wortliste inklusive Doppelnennungen
  und gibt die Werte als dict() zurück"""
  tf_dictionary = dict()
  for word in vocab_list: 
    if word in tf_dictionary: 
      #Eine Kurzschreibweise von tf_dictionary[word] = tf_dictionary[word] + 1
      tf_dictionary[word] += 1
    else: 
      tf_dictionary[word] = 1
  return tf_dictionary
tf = compute_tf(vocab)
print(tf)


{'shipment': 2, 'gold': 2, 'damaged': 1, 'fire': 1, 'delivery': 1, 'silver': 2, 'arrived': 2, 'truck': 2}


Für IDF müssen wir zuerst die Dokumenthäufigkeit berechnen.  

In [124]:
def compute_df(unique_vocab, documents):
  """Berechnet die Dokumenthäufigkeit auf der Basis der in einer Liste gespeicherten Dokumente 
  und einer Liste von Worten ohne Doppelnennungen des gesamten Korpus
  und gibt die Werte als dict() zurück"""
  df_dictionary = dict()
  for word in unique_vocab:
    for document in documents: 
      if word in document:
        if word in df_dictionary: 
          df_dictionary[word] += 1
        else: 
          df_dictionary[word] = 1 
  return df_dictionary

corpus = []
corpus.append(lemmas1)
corpus.append(lemmas2)
corpus.append(lemmas3)
df = compute_df(unique_vocab, corpus)

print(df)

{'silver': 1, 'truck': 2, 'fire': 1, 'arrived': 2, 'shipment': 2, 'damaged': 1, 'delivery': 1, 'gold': 2}


Jetzt haben wir alle Informationen um die inverse Dokumenthäufigkeit zu berechnen:

In [137]:
def compute_idf(df_dictionary):
    """Berechnung der inversen Dokumenthäufigkeit auf Basis der 
    Dokumenthäufigkeit und Anzahl der Dokumente im Korpus
    und gibt die Werte als dict() zurück"""
    idf_dict = {}
    for word in df_dictionary:
      idf_dict[word] = math.log10(len(corpus) / df_dictionary[word])
    return idf_dict
  
idf = compute_idf(df)
print(idf)


{'silver': 0.47712125471966244, 'truck': 0.17609125905568124, 'fire': 0.47712125471966244, 'arrived': 0.17609125905568124, 'shipment': 0.17609125905568124, 'damaged': 0.47712125471966244, 'delivery': 0.47712125471966244, 'gold': 0.17609125905568124}


Und jetzt haben wir alles um TF-IDF zu berechnen.

In [145]:
def compute_tfidf(lemmas1, idf):
  """Berechnung von TFIDF auf Basis der Termhäufigkeit und inversen Dokumenthäufigkeit
  und gibt die berechneten Werte als dict() zurück"""
  tfidf_dict = dict()
  #For each word in the review, we multiply its tf and its idf.
  for word in lemmas1:
    tfidf_dict[word] = lemmas1.count(word) * idf[word]
  return tfidf_dict

tfidf_doc1 = compute_tfidf(lemmas1, idf)
tfidf_doc2 = compute_tfidf(lemmas2, idf)
tfidf_doc3 = compute_tfidf(lemmas3, idf)
print(tfidf_doc1)
print(tfidf_doc2)
print(tfidf_doc3)

{'shipment': 0.17609125905568124, 'gold': 0.17609125905568124, 'damaged': 0.47712125471966244, 'fire': 0.47712125471966244}
{'delivery': 0.47712125471966244, 'silver': 0.9542425094393249, 'arrived': 0.17609125905568124, 'truck': 0.17609125905568124}
{'shipment': 0.17609125905568124, 'gold': 0.17609125905568124, 'arrived': 0.17609125905568124, 'truck': 0.17609125905568124}


**Übung: Berechnen Sie den TF-IDF Wert für einen anderen Korpus - also andere Sätze**

In [0]:
#Ihr Code hier

# Lektion 2: TF-IDF mit bereitgestelleter Library

Anstatt TF-IDF selbst zu programmieren, können wir auch hier eine bereits bestehende Library für maschinelles Lernen namens ´sklearn´ verwenden. Wie Sie sehen wird hier automatisch tokeenisiert und lemmatisiert. Die Stoppwortliste üebrgeben wir bei der Initialisierung des vectorizer um Stoppworte zu entfernen.

In [198]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np

vectorizer = TfidfVectorizer(stop_words=stoplist)
corpus_new = [
              'Shipment of gold damaged in a fire.',
              'Delivery of silver arrived in a silver truck.',
              'Shipment of gold arrived in a truck.']
tfidf_matrix = vectorizer.fit_transform(corpus_new)
wort_index = vectorizer.vocabulary_
idf = vectorizer.idf_
print(wort_index)
print("Dokument, Wortindex, TFID: ", tfidf_matrix)
print("IDF wird hier mit einem anderen Logarithmus berechnet: ", dict(zip(vectorizer.get_feature_names(), idf)))
print("TF-IDF daher auch etwas anders: ", dict(zip(vectorizer.get_feature_names(), np.asarray(tfidf_matrix.sum(axis=0)).ravel())))

{'shipment': 5, 'gold': 4, 'damaged': 1, 'fire': 3, 'delivery': 2, 'silver': 6, 'arrived': 0, 'truck': 7}
Dokument, Wortindex, TFID:    (0, 3)	0.5628290964997665
  (0, 1)	0.5628290964997665
  (0, 4)	0.42804603506311856
  (0, 5)	0.42804603506311856
  (1, 7)	0.3065042162415877
  (1, 0)	0.3065042162415877
  (1, 6)	0.8060324216071015
  (1, 2)	0.40301621080355077
  (2, 7)	0.5
  (2, 0)	0.5
  (2, 4)	0.5
  (2, 5)	0.5
IDF wird hier mit einem anderen Logarithmus berechnet:  {'arrived': 1.2876820724517808, 'damaged': 1.6931471805599454, 'delivery': 1.6931471805599454, 'fire': 1.6931471805599454, 'gold': 1.2876820724517808, 'shipment': 1.2876820724517808, 'silver': 1.6931471805599454, 'truck': 1.2876820724517808}
TF-IDF daher auch etwas anders:  {'arrived': 0.8065042162415876, 'damaged': 0.5628290964997665, 'delivery': 0.40301621080355077, 'fire': 0.5628290964997665, 'gold': 0.9280460350631186, 'shipment': 0.9280460350631186, 'silver': 0.8060324216071015, 'truck': 0.8065042162415876}


# Lektion 3: Clustering

Dieselbe Bibliothek, sklearn, kann auch für Clustering verwendet werden. Dazu verwenden wir die erstellten TF-IDF Vektoren. 


In [222]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

def print_clusters(number_clusters, kmeans):
  order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
  terms = vectorizer.get_feature_names()
  for i in range(number_clusters):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids[i,:3]:
      print(' %s' % terms[ind], end='')
    print()


# Wir müssen die Anzahl an erwarteten Clusterin eingeben - die optimale Zahl ist experimentell zu testen 
number_clusters = 2
kmeans = KMeans(n_clusters=number_clusters, random_state=0).fit(tfidf_matrix)
print_clusters(number_clusters, kmeans)

Cluster 0: shipment gold fire
Cluster 1: silver delivery truck


**Übung: Passen Sie den Algorithmus oben auf Ihre eigenen Sätze an.**

In [0]:
#Ihr Code hier